# Libraries

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
import xgboost

# Importing Data

In [3]:
away_team_stats = pd.read_excel('away_team_stats.xlsx')
home_team_stats = pd.read_excel('home_team_stats.xlsx')
away_team_players = pd.read_excel('awayplayers_final.xlsx')
home_team_players = pd.read_excel('homeplayers_final.xlsx')

In [4]:
data_links = pd.read_excel('data_links.xlsx')
data_links.drop(['Unnamed: 0'], axis=1,inplace=True)

In [5]:
away_team_stats.drop(['Unnamed: 0'], axis=1,inplace=True)
home_team_stats.drop(['Unnamed: 0'], axis=1,inplace=True)
away_team_players.drop(['Unnamed: 0'], axis=1,inplace=True)
home_team_players.drop(['Unnamed: 0'], axis=1,inplace=True)

# Data Cleaning

### transforming date to a suitable variable for the machine learning model

In [6]:
home_team_stats["Date"] = pd.to_datetime(home_team_stats["Date"])
away_team_stats["Date"] = pd.to_datetime(away_team_stats["Date"])

In [7]:
home_team_stats['year'] = home_team_stats['Date'].dt.year
home_team_stats['month'] = home_team_stats['Date'].dt.month
home_team_stats['day'] = home_team_stats['Date'].dt.day
home_team_stats.drop(["Date"], axis=1, inplace=True)

In [8]:
away_team_stats['year'] = away_team_stats['Date'].dt.year
away_team_stats['month'] = away_team_stats['Date'].dt.month
away_team_stats['day'] = away_team_stats['Date'].dt.day
away_team_stats.drop(["Date"], axis=1, inplace=True)

### Creating the main table for the machine learning model

In [9]:
overall_stats_1= home_team_stats['Full Time Home Team Goals'] - away_team_stats['Full Time Away Team Goals']
overall_stats_2= home_team_stats['Half Time Home Team Goals'] - away_team_stats['Half Time Away Team Goals']
overall_stats_3= home_team_stats['Home Team Shots'] - away_team_stats['Away Team Shots']
overall_stats_4= home_team_stats['Home Team Shots on Target'] - away_team_stats['Away Team Shots on Target']
overall_stats_5= home_team_stats['Home Team Fouls Committed'] - away_team_stats['Away Team Fouls Committed']
overall_stats_6= home_team_stats['Home Team Corners'] - away_team_stats['Away Team Corners']
overall_stats_7= home_team_stats['Home Team Yellow Cards'] - away_team_stats['Away Team Yellow Cards']
overall_stats_8= home_team_stats['Home Team Red Cards'] - away_team_stats['Away Team Red Cards']

In [10]:
overall_stats = pd.DataFrame({'Full Time Goals':overall_stats_1,'Half Time Goals':overall_stats_2,'Shots':overall_stats_3,'Shots on Target':overall_stats_4,'Fouls Committed':overall_stats_5,'Corners':overall_stats_6,'Yellow Cards':overall_stats_7,'Red Cards':overall_stats_8})

### Encode a result column from the full time goals

In [11]:
overall_stats['Full Time Goals'] = pd.cut(overall_stats['Full Time Goals'], bins=[-100,-0.1,0.1,100], labels=["Loss", "Draw", "Win"])

In [12]:
overall_stats.rename(columns = {'Full Time Goals':'Result'}, inplace=True)

In [13]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
x = overall_stats['Result']
overall_stats['Result'] = label_encoder.fit_transform(x)
display(overall_stats)

,Result,Half Time Goals,Shots,Shots on Target,Fouls Committed,Corners,Yellow Cards,Red Cards
0,1,-1,0,-7,-1,3,1,0
1,0,1,-6,-2,-4,-4,-2,1
2,0,-1,-1,-1,-6,2,3,0
3,2,-1,7,2,4,5,0,0
4,2,1,1,2,-6,-4,0,0
...,...,...,...,...,...,...,...,...
1102,2,1,5,-1,7,5,1,0
1103,2,-1,1,0,3,5,3,0
1104,1,-1,-4,-1,1,1,1,0
1105,2,0,3,2,-1,2,-1,0


# Model

In [14]:
X = overall_stats.drop(['Result'], axis=1)
y = overall_stats[['Result']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 1)

## Decision Tree

In [15]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth = 3)
model.fit(X_train, y_train)

# Score the model  
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))
#Doesn't overfit

0.7111913357400722
0.6578313253012048


In [16]:
model.predict(X_test)

array([2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 0, 1, 2, 2, 2, 2, 2,
       1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2,
       1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 0,
       2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 2, 1, 1, 2,
       2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 0, 2,
       2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1,
       2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2,
       2, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1,
       2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2,
       2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 0, 1, 1, 1, 2, 2, 2,
       2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2,
       2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2])

#### Pickling the model

In [17]:
import pickle

pickle.dump(model, open('model.p', 'wb'))

In [18]:
X_test.to_pickle('xtest.p')
y_test.to_pickle('ytest.p')

## Preparing for the predictions

### Create a table with team stats

In [19]:
stats_by_hometeam=pd.concat([home_team_stats,data_links['Home team']], axis=1)
stats_by_awayteam=pd.concat([away_team_stats,data_links['Away team']], axis=1)

In [20]:
stats_by_hometeam.rename(columns = {'Full Time Home Team Goals':'Full Time Goals','Half Time Home Team Goals':'Half Time Goals','Home Team Shots':'Shots','Home Team Shots on Target':'Shots on Target','Home Team Fouls Committed':'Fouls Committed','Home Team Corners':'Corners','Home Team Yellow Cards':'Yellow Cards','Home Team Red Cards':'Red Cards'},inplace=True)
stats_by_awayteam.rename(columns = {'Full Time Away Team Goals':'Full Time Goals','Half Time Away Team Goals':'Half Time Goals','Away Team Shots':'Shots','Away Team Shots on Target':'Shots on Target','Away Team Fouls Committed':'Fouls Committed','Away Team Corners':'Corners','Away Team Yellow Cards':'Yellow Cards','Away Team Red Cards':'Red Cards'},inplace=True)


In [21]:
stats_by_hometeam.rename(columns = {'Home team':'Team'},inplace=True)
stats_by_awayteam.rename(columns = {'Away team':'Team'},inplace=True)

In [22]:
stats_by_team=pd.concat([stats_by_hometeam,stats_by_awayteam], axis=0)
stats_by_team

,Full Time Goals,Half Time Goals,Shots,Shots on Target,Fouls Committed,Corners,Yellow Cards,Red Cards,year,month,day,Team
0,0,0,12,2,15,6,1,0,2017,6,8,CD Aves
1,1,1,6,2,18,4,1,1,2017,6,8,V. Setúbal
2,1,0,12,5,19,5,4,0,2017,7,8,Portimonense
3,2,0,12,4,14,9,1,0,2017,7,8,Feirense
4,1,1,11,5,17,4,2,0,2017,7,8,Rio Ave
...,...,...,...,...,...,...,...,...,...,...,...,...
1102,1,0,12,5,14,3,3,0,2020,7,25,FC Porto
1103,1,1,10,5,13,3,3,0,2020,7,25,Sporting
1104,2,1,14,6,18,4,2,0,2020,7,26,CD Tondela
1105,0,0,10,3,20,4,3,0,2020,7,26,Belenenses SAD


In [23]:
stats_by_team = stats_by_hometeam.groupby(['Team']).agg({'Half Time Goals':'mean','Shots':'mean','Shots on Target':'mean','Fouls Committed':'mean','Corners':'mean','Yellow Cards':'mean','Red Cards':'mean'})
stats_by_team

,Half Time Goals,Shots,Shots on Target,Fouls Committed,Corners,Yellow Cards,Red Cards
Team,,,,,,,
Belenenses,0.588235,8.764706,4.176471,16.529412,4.941176,2.647059,0.117647
Belenenses SAD,0.600000,9.577778,3.266667,16.755556,4.800000,2.711111,0.155556
Benfica,0.645161,13.693548,5.290323,15.516129,6.500000,1.790323,0.096774
Boavista,0.606557,11.754098,4.032787,18.163934,5.278689,2.688525,0.131148
CD Aves,0.509804,10.941176,3.882353,16.470588,5.274510,2.588235,0.137255
CD Tondela,0.451613,10.838710,3.822581,15.951613,5.080645,2.177419,0.193548
Estoril Praia,0.941176,11.823529,5.294118,13.764706,6.058824,1.764706,0.117647
FC Famalicão,0.571429,12.785714,4.785714,16.107143,5.071429,2.750000,0.142857
FC Porto,1.096774,14.854839,6.032258,16.209677,6.338710,1.887097,0.080645


### Create a table with players and stats

#### Home

In [24]:
home_team_stats.rename(columns = {'Full Time Home Team Goals':'Full Time Goals','Half Time Home Team Goals':'Half Time Goals','Home Team Shots':'Shots','Home Team Shots on Target':'Shots on Target','Home Team Fouls Committed':'Fouls Committed','Home Team Corners':'Corners','Home Team Yellow Cards':'Yellow Cards','Home Team Red Cards':'Red Cards'},inplace=True)
away_team_stats.rename(columns = {'Full Time Away Team Goals':'Full Time Goals','Half Time Away Team Goals':'Half Time Goals','Away Team Shots':'Shots','Away Team Shots on Target':'Shots on Target','Away Team Fouls Committed':'Fouls Committed','Away Team Corners':'Corners','Away Team Yellow Cards':'Yellow Cards','Away Team Red Cards':'Red Cards'},inplace=True)

In [25]:
home_team_players.columns = home_team_players.columns.str.rstrip('\xa0(C)') 
home_team_players_T = home_team_players.T.groupby(level=0).mean()
home_team_players = home_team_players_T.T
home_team_players

,Pedrão,Abdel Medioub,Abdoulay Diaby,Abdoulaye Ba,Abdu Conté,Abdul Mumin,Abel Ruiz,Abner,Abner Felipe,Accioly,...,Álex Centelles,Álex Grimaldo,Ângelo Neto,Éber Bessa,Éder Militão,Éverton Nascimento,Ícaro Silva,Óliver Torres,Óscar Barreto,Óscar Estupiñán
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1104,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Away

In [26]:
away_team_players.columns = away_team_players.columns.str.rstrip('\xa0(C)') 
away_team_players_T = away_team_players.T.groupby(level=0).mean()
away_team_players = away_team_players_T.T
away_team_players

,Pedrão,Aaron Tshibola,Abdel Medioub,Abdoulay Diaby,Abdoulaye Ba,Abdu Conté,Abdul Mumin,Abel Ruiz,Ablaye Faye,Abner,...,Álex Centelles,Álex Grimaldo,Ângelo Neto,Éber Bessa,Éder Militão,Éverton Nascimento,Ícaro Silva,Óliver Torres,Óscar Barreto,Óscar Estupiñán
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1103,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Joinning home and away

In [27]:
team_players=pd.concat([home_team_players,away_team_players], axis=0)
team_stats=pd.concat([home_team_stats,away_team_stats], axis=0)
team_stats

,Full Time Goals,Half Time Goals,Shots,Shots on Target,Fouls Committed,Corners,Yellow Cards,Red Cards,year,month,day
0,0,0,12,2,15,6,1,0,2017,6,8
1,1,1,6,2,18,4,1,1,2017,6,8
2,1,0,12,5,19,5,4,0,2017,7,8
3,2,0,12,4,14,9,1,0,2017,7,8
4,1,1,11,5,17,4,2,0,2017,7,8
...,...,...,...,...,...,...,...,...,...,...,...
1102,1,0,12,5,14,3,3,0,2020,7,25
1103,1,1,10,5,13,3,3,0,2020,7,25
1104,2,1,14,6,18,4,2,0,2020,7,26
1105,0,0,10,3,20,4,3,0,2020,7,26


In [28]:
HTG = team_players.replace(0, np.nan).multiply(team_stats['Half Time Goals'], axis="index").mean()
S = team_players.replace(0, np.nan).multiply(team_stats['Shots'], axis="index").mean()
SOT = team_players.replace(0, np.nan).multiply(team_stats['Shots on Target'], axis="index").mean()
FC = team_players.replace(0, np.nan).multiply(team_stats['Fouls Committed'], axis="index").mean()
C = team_players.replace(0, np.nan).multiply(team_stats['Corners'], axis="index").mean()
YC = team_players.replace(0, np.nan).multiply(team_stats['Yellow Cards'], axis="index").mean()
RC = team_players.replace(0, np.nan).multiply(team_stats['Red Cards'], axis="index").mean()

players_stats_df = pd.DataFrame(data= {'Half Time Goals': HTG, 'Shots': S, 'Shots on Target': SOT, 'Fouls Committed': FC, 'Corners': C, 'Yellow Cards': YC, 'Red Cards': RC})

## Making a list of players for each team

#### Home

In [29]:
players_of_each_hometeam_1 = pd.concat([home_team_players,data_links['Home team']], axis=1)
players_of_each_hometeam = pd.melt(players_of_each_hometeam_1, id_vars=['Home team'])
players_of_each_hometeam = players_of_each_hometeam[players_of_each_hometeam.value != 0]
players_of_each_hometeam = players_of_each_hometeam.drop_duplicates(subset=["variable"])
display(players_of_each_hometeam)

,Home team,variable,value
902,Nacional,Pedrão,1.0
2062,CD Tondela,Abdel Medioub,1.0
2585,Sporting,Abdoulay Diaby,1.0
4388,Moreirense,Abdoulaye Ba,1.0
5054,Moreirense,Abdu Conté,1.0
...,...,...,...
1097076,Marítimo,Éverton Nascimento,1.0
1098326,CD Tondela,Ícaro Silva,1.0
1099257,FC Porto,Óliver Torres,1.0
1100362,Rio Ave,Óscar Barreto,1.0


#### Away

In [30]:
players_of_each_awayteam_1 = pd.concat([away_team_players,data_links['Away team']], axis=1)
players_of_each_awayteam = pd.melt(players_of_each_awayteam_1, id_vars=['Away team'])
players_of_each_awayteam = players_of_each_awayteam[players_of_each_awayteam.value != 0]
players_of_each_awayteam = players_of_each_awayteam.drop_duplicates(subset=["variable"])
display(players_of_each_awayteam)

,Away team,variable,value
912,Nacional,Pedrão,1.0
1937,CD Aves,Aaron Tshibola,1.0
3189,CD Tondela,Abdel Medioub,1.0
3698,Sporting,Abdoulay Diaby,1.0
5474,Moreirense,Abdoulaye Ba,1.0
...,...,...,...
1132496,Marítimo,Éverton Nascimento,1.0
1133756,CD Tondela,Ícaro Silva,1.0
1134690,FC Porto,Óliver Torres,1.0
1135794,Rio Ave,Óscar Barreto,1.0


#### Joining Home and Away

In [37]:
players_of_each_awayteam.rename(columns = {'Away team':'Team'},inplace=True)
players_of_each_hometeam.rename(columns = {'Home team':'Team'},inplace=True)

players_of_each_team_tableau = pd.concat([players_of_each_awayteam,players_of_each_hometeam], axis=0)
players_of_each_team = players_of_each_team_tableau.drop_duplicates(subset=["variable"])
players_of_each_team = players_of_each_team.groupby('Team')['variable'].apply(list).reset_index(name='List of players')
players_of_each_team.set_index('Team', inplace=True)
display(players_of_each_team)

,List of players
Team,
Belenenses,"[André Geraldes, André Moreira, André Sousa, B..."
Belenenses SAD,"[Afonso Sousa, Afonso Taira, Alhassane Keita, ..."
Benfica,"[Adel Taarabt, Andreas Samaris, Andrija Zivkov..."
Boavista,"[Adil Rami, Alberth Elis, Alberto Bueno, Aleja..."
CD Aves,"[Aaron Tshibola, Ablaye Faye, Adam Dzwigala, A..."
CD Tondela,"[Abdel Medioub, Babacar Niasse, Bruno Monteiro..."
Estoril Praia,"[Abner, Ailton Silva, Allano, André Claro, Ayl..."
FC Famalicão,"[Anderson Oliveira, Andrija Lukovic, Bruno Jor..."
FC Porto,"[Adrián López, Agustín Marchesín, Alex Telles,..."


## Loading dataset into files

In [38]:
players_stats_df.to_excel("players_stats_df.xlsx")
players_of_each_team.to_excel("players_of_each_team.xlsx")
stats_by_team.to_excel("stats_by_team.xlsx")
players_of_each_team_tableau.to_excel("players_of_each_team_tableau.xlsx")

# MAIN PREDICTION CODE!!!

In [33]:
def choose_team():
    buildteam_stats={}
    teams=' / '.join(players_of_each_team.index.tolist())
    x=True
    z=True


    while z:

        select = input(f'Choose the team you are going to play against: {teams}: ')
        if select in teams:
            opposing_team = stats_by_team.loc[select].values
            z= False

        else:
            print("This team doesn't exist, please choose another one.")
            continue

    print('')

    while x:

        select = input(f'Choose your team: {teams}')
        if select in teams:
            team = players_of_each_team.loc[select,'List of players']
            x = False

        else:
            print("That team doesn't exist, choose again.")
            continue

    display('These are the players from your team:',team)

    print('')

    for i in list(range(0,11)):
        print('')
        print('Choose player number',i,': ')
        y=True
        while y:
            select = input(f'Choose a player: ')
            if select in team:
                player = players_stats_df.loc[select]
                y = False

                buildteam_stats[select]=player.values
            else:
                print("This player doesn't exist, please choose again.")
                print('')
                continue


    print('')  

    team_power = np.array([l.tolist() for l in buildteam_stats.values()]).mean(0)

    if model.predict([team_power - opposing_team]) == np.array(0):
        print('With this squad your team will DRAW the game!')
    elif model.predict([team_power - opposing_team]) == np.array(1):
        print('With this squad your team will LOSE the game!')
    elif model.predict([team_power - opposing_team]) == np.array(2):
        print('With this squad your team will WIN the game!')

    print('') 
    print('These are the stats of each player of your squad: ')    
    print(players_stats_df.loc[buildteam_stats.keys()])

In [34]:
print(choose_team())

KeyboardInterrupt: Interrupted by user